In [2]:
import pandas as pd
import cvxpy as cp

# Load data from the files
generators = pd.read_csv('/Users/skyegoodman/Documents/OPTIMISING POWER GRIDS/Aurora-Energy/generators.csv')
demand_data = pd.read_csv('/Users/skyegoodman/Documents/OPTIMISING POWER GRIDS/Aurora-Energy/hourlydemandbynode.csv')


# Extract generator parameters
marginal_costs = generators['MC'].values  # Marginal costs (£/MWh)
capacities = generators['CAP'].values    # Maximum capacities (MW)

# Calculate total hourly demand
total_hourly_demand = demand_data.sum(axis=1).values  # Sum across all nodes for each hour

# Number of generators and time periods
num_generators = len(marginal_costs)
num_hours = len(total_hourly_demand)

# Define decision variables
generation = cp.Variable((num_hours, num_generators))  # Generation for each generator and hour

# Define the objective function (minimize total generation cost)
objective = cp.Minimize(cp.sum(cp.multiply(marginal_costs, cp.sum(generation, axis=0))))

# Define constraints
constraints = []

# Constraint 1: Meet the total hourly demand
for t in range(num_hours):
    constraints.append(cp.sum(generation[t, :]) == total_hourly_demand[t])

# Constraint 2: Generator capacity limits
constraints.extend([generation[:, i] >= 0 for i in range(num_generators)])  # Non-negativity
constraints.extend([generation[:, i] <= capacities[i] for i in range(num_generators)])  # Max capacity

# Solve the optimization problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS)

# Check the results
if problem.status == cp.OPTIMAL:
    print(f"Optimal total generation cost: £{problem.value:.2f}")
    optimal_generation = generation.value
    for i in range(num_generators):
        print(f"Generator {i+1} output over time: {optimal_generation[:, i]}")
else:
    print("Problem could not be solved to optimality.")


Optimal total generation cost: £2931427.30
Generator 1 output over time: [234.96636704 303.98002276 299.30475881 298.03553341 302.3152824
 313.13970993 325.6453737  335.25231294 348.60877337 349.99999992
 349.99999992 349.99999992 349.99999992 349.99999992 349.99999992
 349.99999992 349.99999992 349.99999991 349.99999996 349.99999996
 349.99999996 349.99999996 349.99999993 349.99999992 349.99999992
 349.99999992 349.99999992 349.99999992 349.99999992 349.99999996
 349.99999997 350.         350.         350.         350.
 350.         350.         349.99999996 349.99999997 349.99999997
 349.99999997 350.         350.         350.         350.
 350.         350.         349.99999997 349.99999997 349.99999996
 349.99999996 349.99999997 349.99999997 350.         350.
 350.         350.         350.         350.         349.99999997
 349.99999997 349.99999996 349.99999996 349.99999991 349.99999996
 349.99999997 350.         350.         350.         349.99999996
 349.99999996 349.99999992 3